In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Cystic_Fibrosis"
cohort = "GSE76347"

# Input paths
in_trait_dir = "../../input/GEO/Cystic_Fibrosis"
in_cohort_dir = "../../input/GEO/Cystic_Fibrosis/GSE76347"

# Output paths
out_data_file = "../../output/preprocess/Cystic_Fibrosis/GSE76347.csv"
out_gene_data_file = "../../output/preprocess/Cystic_Fibrosis/gene_data/GSE76347.csv"
out_clinical_data_file = "../../output/preprocess/Cystic_Fibrosis/clinical_data/GSE76347.csv"
json_path = "../../output/preprocess/Cystic_Fibrosis/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import re
import os
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains nasal epithelial cell samples analyzed for microarray
# analysis, which suggests gene expression data, not just miRNA or methylation data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Cystic Fibrosis):
# From sample characteristics, all patients have CF (row 0: 'disease state: CF')
# Since everyone has CF, we can't do a case-control study within this dataset alone
# But we can still use this dataset for gene expression analysis related to CF
trait_row = 0  # Everyone has CF, so this is a constant feature but we still record it

# For age:
# No age information is available in the sample characteristics
age_row = None  # Age data not available

# For gender:
# No gender information is available in the sample characteristics
gender_row = None  # Gender data not available

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert trait data to binary format (1 for CF, 0 for non-CF)."""
    if value is None:
        return None
    # Extract the value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    # If the value indicates CF, return 1 (all patients in this study have CF)
    if value.lower() == "cf":
        return 1
    return None  # For any other value, return None

def convert_age(value):
    """Convert age data to continuous format."""
    # This function is included for completeness but won't be used since age_row is None
    if value is None:
        return None
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender data to binary format (0 for female, 1 for male)."""
    # This function is included for completeness but won't be used since gender_row is None
    if value is None:
        return None
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    if value.lower() in ["female", "f"]:
        return 0
    elif value.lower() in ["male", "m"]:
        return 1
    return None

# 3. Save Metadata
# Determine trait data availability (all have CF, so trait data is available)
is_trait_available = trait_row is not None

# Conduct initial filtering on dataset usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Create a properly structured clinical data DataFrame
# We need to create a DataFrame where each row corresponds to a clinical feature
# and each column corresponds to a sample

# First, get the sample characteristics dictionary
sample_characteristics_dict = {
    0: ['disease state: CF'], 
    1: ['individual: patient # 001', 'individual: patient # 002', 'individual: patient # 004', 'individual: patient # 006', 
        'individual: patient # 009', 'individual: patient # 013', 'individual: patient # 015', 'individual: patient # 017', 
        'individual: patient # 019', 'individual: patient # 020', 'individual: patient # 021', 'individual: patient # 024', 
        'individual: patient # 025', 'individual: patient # 028', 'individual: patient # 030', 'individual: patient # 031', 
        'individual: patient # 003', 'individual: patient # 005', 'individual: patient # 010', 'individual: patient # 014', 
        'individual: patient # 018', 'individual: patient # 022', 'individual: patient # 027'],
    2: ['treatment: digitoxin', 'treatment: placebo'],
    3: ['dosage: 50 micro gram/daily', 'dosage: 100 micro gram/daily'],
    4: ['time: post treatment', 'time: pre treatment'],
    5: ['cell type: nasal epithelial cells']
}

# Create a DataFrame that represents the structure expected by geo_select_clinical_features
# The function expects rows as features, not directly from the sample characteristics dict
clinical_data = pd.DataFrame()
for row_idx, values in sample_characteristics_dict.items():
    clinical_data.loc[row_idx, 0] = values[0] if values else None

# Since trait_row is not None, we extract the clinical features
if trait_row is not None:
    # Extract clinical features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the data
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


### Step 4: Gene Identifier Review

In [ ]:
# Examine the identifiers in the first few rows of the gene expression data
# The IDs like '2315100' appear to be numeric identifiers that are not standard gene symbols
# These are likely probe IDs from a microarray platform that need to be mapped to gene symbols

# Standard human gene symbols follow patterns like BRCA1, TP53, etc.
# The numeric identifiers seen in this dataset (2315100, 2315106, etc.) are not recognizable gene symbols

# Since these are numeric identifiers rather than human gene symbols, 
# they will require mapping to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify columns for mapping
# From the previous output, we can see:
# - 'ID' column contains the probe identifiers (matching the format in gene_data index)
# - 'gene_assignment' column contains information about gene symbols

# Create a gene mapping dataframe directly using the get_gene_mapping function
# The 'ID' column contains the probe identifiers and the 'gene_assignment' column contains gene symbol information
mapping_df = get_gene_mapping(gene_annotation, 'ID', 'gene_assignment')

# Preview the mapping dataframe
print("Gene mapping preview:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
# Use the apply_gene_mapping function from the library
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print information about the resulting gene expression data
print(f"\nGene expression data shape after mapping: {gene_data.shape}")
print("\nFirst few gene symbols:")
print(gene_data.index[:5])

# Normalize gene symbols to ensure consistency
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"\nGene expression data shape after normalization: {gene_data.shape}")
print("\nFirst few normalized gene symbols:")
print(gene_data.index[:5])

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the gene expression data
gene_data.to_csv(out_gene_data_file)
print(f"\nGene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. For step 1, we'll skip normalizing gene symbols again since it was already done in the previous step
# and the normalized gene data is already saved

# 2. Load the saved clinical data instead of reprocessing it
clinical_data_path = out_clinical_data_file
if os.path.exists(clinical_data_path):
    selected_clinical_df = pd.read_csv(clinical_data_path)
    print(f"Loaded clinical data from {clinical_data_path}")
    print("Clinical data preview:")
    print(preview_df(selected_clinical_df))
else:
    # If the file doesn't exist for some reason, extract clinical features as a fallback
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    selected_clinical_df = geo_select_clinical_features(
        clinical_data, 
        trait=trait,  # Use the trait variable from setup
        trait_row=trait_row, 
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    print("Clinical data preview:")
    print(preview_df(selected_clinical_df))

# Load the normalized gene data 
gene_data_path = out_gene_data_file
if os.path.exists(gene_data_path):
    normalized_gene_data = pd.read_csv(gene_data_path, index_col=0)
    print(f"Loaded normalized gene data from {gene_data_path}")
    print(f"Normalized gene data shape: {normalized_gene_data.shape}")
else:
    print("Error: Normalized gene data file not found")
    
# 3. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape (before handling missing values): {linked_data.shape}")

# 4. Handle missing values in the linked data
# Determine the actual column name for the trait
trait_column = '0'  # Based on the clinical data preview
linked_data = handle_missing_values(linked_data, trait_column)
print(f"Data after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are biased
# Check if trait is biased
if len(linked_data[trait_column].unique()) == 2:  # binary trait
    is_trait_biased = judge_binary_variable_biased(linked_data, trait_column)
else:  # continuous trait
    is_trait_biased = judge_continuous_variable_biased(linked_data, trait_column)

# We don't need to do any further processing for demographic variables since none exist in this dataset
unbiased_linked_data = linked_data  # No biased features to remove

# 6. Conduct final quality validation and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data from nasal epithelial cells of cystic fibrosis patients in a study examining the effects of digitoxin on airway inflammation."
)

# 7. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical features using the functions defined in step 2
# First, let's load the clinical data again to ensure we have the latest version
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Extract clinical features using melanoma vs normal tissue as the binary trait
selected_clinical_df = geo_select_clinical_features(
    clinical_data, 
    trait="Melanoma", 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")
print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# 3. Link the clinical and genetic data
# Transpose normalized gene data for linking
gene_data_t = normalized_gene_data.T

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape (before handling missing values): {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, "Melanoma")
print(f"Data after handling missing values: {linked_data.shape}")

# 5. Determine whether the trait and demographic features are biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, "Melanoma")

# 6. Conduct final quality validation and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression data comparing melanoma (primary and metastatic) with normal tissue/nevi."
)

# 7. If the linked data is usable, save it
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")